In [2]:
import random
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt
import cloudpickle as cpkl
from pathlib import Path
import numpy as np
import datetime
from datetime import timedelta
from functools import reduce
from fbprophet import Prophet
import multiprocessing as mp
from itertools import repeat
from joblib import Parallel, delayed
import calendar

In [3]:
script_name='prophet_item_store'
train_pkl_file = 'input/train_pkl'
test_pkl_file = 'input/test_pkl'
train,transactions,stores,oil,items,holidays_events = cpkl.loads(open(train_pkl_file,'rb').read())
test = cpkl.loads(open(test_pkl_file,'rb').read())

In [4]:
big_bang = datetime.date(2014,12,31)
maxdate = datetime.date(2017,8,31)
train = train.loc[train.date>big_bang]
#train.loc[:, "onpromotion"].fillna(False, inplace=True)
transactions = transactions.loc[transactions.date>big_bang]

In [5]:
holidays_events.date = pd.to_datetime(holidays_events.date)
holidays_events.date = holidays_events.date.dt.date
all_days = pd.DataFrame({'date':pd.date_range(big_bang,maxdate,freq="D")})
all_days.date = all_days.date.dt.date
all_days['holiday']=0
all_days.loc[all_days.date.isin(holidays_events.date),'holiday']=1
train.date = train.date.dt.date
transactions.date = transactions.date.dt.date
oil.date = oil.date.dt.date

In [6]:
holidays_events = holidays_events[holidays_events.transferred==False]
print(holidays_events.head())
print(all_days.head())

         date     type    locale locale_name                    description  \
0  2012-03-02  Holiday     Local       Manta             Fundacion de Manta   
1  2012-04-01  Holiday  Regional    Cotopaxi  Provincializacion de Cotopaxi   
2  2012-04-12  Holiday     Local      Cuenca            Fundacion de Cuenca   
3  2012-04-14  Holiday     Local    Libertad      Cantonizacion de Libertad   
4  2012-04-21  Holiday     Local    Riobamba      Cantonizacion de Riobamba   

   transferred  
0        False  
1        False  
2        False  
3        False  
4        False  
         date  holiday
0  2014-12-31        1
1  2015-01-01        1
2  2015-01-02        1
3  2015-01-03        0
4  2015-01-04        0


In [7]:
train.onpromotion = train.onpromotion.astype(int)
print(train.shape)
print(train.tail())


(86902776, 6)
                  id        date  store_nbr  item_nbr  unit_sales  onpromotion
125497035  125497035  2017-08-15         54   2089339         4.0            0
125497036  125497036  2017-08-15         54   2106464         1.0            1
125497037  125497037  2017-08-15         54   2110456       192.0            0
125497038  125497038  2017-08-15         54   2113914       198.0            1
125497039  125497039  2017-08-15         54   2116416         2.0            0


In [8]:
oil.dcoilwtico.fillna(method='ffill',inplace=True)
print(oil.head())
import gc
gc.collect()

         date  dcoilwtico
0  2013-01-01         NaN
1  2013-01-02       93.14
2  2013-01-03       92.97
3  2013-01-04       93.12
4  2013-01-07       93.20


252

In [9]:
class elapsed(object):
    def __init__(self, fld):
        self.fld = fld
        self.last = pd.to_datetime(np.nan)
        self.last_store = 0
        self.last_item = 0
        
    def get(self, row):
        if (row.store_nbr != self.last_store) or (row.item_nbr != self.last_item):
            self.last = pd.to_datetime(np.nan)
            self.last_store = row.store_nbr
            self.last_item = row.item_nbr
        if (row[self.fld]): self.last = row.date
        return (row.date-self.last).days

class elapsed_date(object):
    def __init__(self, fld):
        self.fld = fld
        self.last = pd.to_datetime(np.nan)
        
    def get(self, row):
        if (row[self.fld]): self.last = row.date
        return (row.date-self.last).days
gc.collect()


41

In [10]:
def add_elapsed_date(fld, prefix):
    sh_el = elapsed_date(fld)
    all_days[prefix+fld] = all_days.apply(sh_el.get, axis=1)
fld='holiday'
all_days = all_days.sort_values(['date'])
add_elapsed_date(fld, 'After')
all_days = all_days.sort_values([ 'date'], ascending=[False])
add_elapsed_date(fld, 'Before')
all_days.loc[all_days.date==datetime.date(2017,8,25),'Beforeholiday']=-34
all_days.loc[all_days.date==datetime.date(2017,8,26),'Beforeholiday']=-33
all_days.loc[all_days.date==datetime.date(2017,8,27),'Beforeholiday']=-32
all_days.loc[all_days.date==datetime.date(2017,8,28),'Beforeholiday']=-31
all_days.loc[all_days.date==datetime.date(2017,8,29),'Beforeholiday']=-30
all_days.loc[all_days.date==datetime.date(2017,8,30),'Beforeholiday']=-29
all_days.loc[all_days.date==datetime.date(2017,8,31),'Beforeholiday']=-28
#print(all_days[all_days.date>datetime.date(2016,8,23)])
print(all_days)

           date  holiday  Afterholiday  Beforeholiday
974  2017-08-31        0             7          -28.0
973  2017-08-30        0             6          -29.0
972  2017-08-29        0             5          -30.0
971  2017-08-28        0             4          -31.0
970  2017-08-27        0             3          -32.0
969  2017-08-26        0             2          -33.0
968  2017-08-25        0             1          -34.0
967  2017-08-24        1             0            0.0
966  2017-08-23        0             8           -1.0
965  2017-08-22        0             7           -2.0
964  2017-08-21        0             6           -3.0
963  2017-08-20        0             5           -4.0
962  2017-08-19        0             4           -5.0
961  2017-08-18        0             3           -6.0
960  2017-08-17        0             2           -7.0
959  2017-08-16        0             1           -8.0
958  2017-08-15        1             0            0.0
957  2017-08-14        0    

In [11]:
min(all_days.Beforeholiday)

-55.0

In [12]:
columns = ["date", "store_nbr","item_nbr" ,"onpromotion"]
# try merging all_days with train here to fill missing dates to get the counters on onpromotion right
df = train[columns]
def add_elapsed(fld, prefix):
    sh_el = elapsed(fld)
    df[prefix+fld] = df.apply(sh_el.get, axis=1)
print(df.head())

                date  store_nbr  item_nbr  onpromotion
38594264  2015-01-01         25    103665            0
38594265  2015-01-01         25    105575            0
38594266  2015-01-01         25    108634            0
38594267  2015-01-01         25    108698            0
38594268  2015-01-01         25    108786            0


In [13]:
df.sort_values(['store_nbr','item_nbr','date'],inplace=True)
fld = 'onpromotion'
add_elapsed(fld, 'After')
df.sort_values(['store_nbr','item_nbr', 'date'], ascending=[True,True,False],inplace=True)
add_elapsed(fld, 'Before')

/home/suresh/anaconda3/envs/kag_unitforecast/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/suresh/anaconda3/envs/kag_unitforecast/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/suresh/anaconda3/envs/kag_unitforecast/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/s

In [14]:
df.Afteronpromotion.fillna(0,inplace=True)
df.Beforeonpromotion.fillna(0,inplace=True)

In [15]:
#df.reset_index(inplace=True)
df = df[["date","store_nbr","item_nbr","onpromotion","Afteronpromotion","Beforeonpromotion"]]
df.head()

,date,store_nbr,item_nbr,onpromotion,Afteronpromotion,Beforeonpromotion
124877031,2017-08-10,1,96995,0,0.0,0.0
124571010,2017-08-07,1,96995,0,0.0,0.0
124460168,2017-08-06,1,96995,0,0.0,0.0
124142866,2017-08-03,1,96995,0,0.0,0.0
124035459,2017-08-02,1,96995,0,0.0,0.0


In [16]:
if False:
    te = df[(df.store_nbr==54) & (df.item_nbr==2106464)]
    te.set_index('date',inplace=True)
    print(te)
    #df.set_index('date',inplace=True)
    #bwd = df[['store_nbr','item_nbr','date','onpromotion']].sort_values(['store_nbr','item_nbr','date']).groupby(['store_nbr','item_nbr']).rolling(7,min_periods=1).sum().reset_index()
    bwd = te[['store_nbr','item_nbr','onpromotion']].sort_index().groupby(['store_nbr','item_nbr']).rolling(7,min_periods=1).sum()
    bwd.drop(['store_nbr','item_nbr'],1,inplace=True)
    bwd.reset_index(inplace=True)
    #print(bwd)
    fwd = te[['store_nbr','item_nbr','onpromotion']].sort_index(ascending=False).groupby(['store_nbr','item_nbr']).rolling(7,min_periods=1).sum()
    fwd.drop(['store_nbr','item_nbr'],1,inplace=True)
    fwd.reset_index(inplace=True)
    #print(fwd)
    te.reset_index(inplace=True)
    te =te.merge(fwd,'left',['store_nbr','item_nbr','date'],suffixes=['','_fw'],copy=False)
    te =te.merge(bwd,'left',['store_nbr','item_nbr','date'],suffixes=['','_bw'],copy=False)
    print(te)

In [17]:
df.set_index('date',inplace=True)


In [18]:
df.head()

,store_nbr,item_nbr,onpromotion,Afteronpromotion,Beforeonpromotion
date,,,,,
2017-08-10,1,96995,0,0.0,0.0
2017-08-07,1,96995,0,0.0,0.0
2017-08-06,1,96995,0,0.0,0.0
2017-08-03,1,96995,0,0.0,0.0
2017-08-02,1,96995,0,0.0,0.0


In [19]:
bwd = df[['store_nbr','item_nbr','onpromotion']].sort_index().groupby(['store_nbr','item_nbr']).rolling(7,min_periods=1).sum()
bwd.drop(['store_nbr','item_nbr'],1,inplace=True)
bwd.reset_index(inplace=True)

In [20]:
fwd = df[['store_nbr','item_nbr','onpromotion']].sort_index(ascending=False).groupby(['store_nbr','item_nbr']).rolling(7,min_periods=1).sum()
fwd.drop(['store_nbr','item_nbr'],1,inplace=True)
fwd.reset_index(inplace=True)
df.reset_index(inplace=True)

In [21]:
df = df.merge(bwd,on=['date','store_nbr','item_nbr'],how='left',suffixes=['','_bw'], copy=False)
df = df.merge(fwd,on=['date','store_nbr','item_nbr'],how='left',suffixes=['','_fw'], copy=False)
df.head()

,date,store_nbr,item_nbr,onpromotion,Afteronpromotion,Beforeonpromotion,onpromotion_bw,onpromotion_fw
0,2017-08-10,1,96995,0,0.0,0.0,0.0,0.0
1,2017-08-07,1,96995,0,0.0,0.0,0.0,0.0
2,2017-08-06,1,96995,0,0.0,0.0,0.0,0.0
3,2017-08-03,1,96995,0,0.0,0.0,0.0,0.0
4,2017-08-02,1,96995,0,0.0,0.0,0.0,0.0


In [22]:
df.loc[(df.store_nbr==54) & (df.item_nbr==2106464)]

,date,store_nbr,item_nbr,onpromotion,Afteronpromotion,Beforeonpromotion,onpromotion_bw,onpromotion_fw
86902576,2017-08-15,54,2106464,1,0.0,0.0,2.0,1.0
86902577,2017-08-14,54,2106464,1,0.0,0.0,1.0,2.0
86902578,2017-08-08,54,2106464,0,35.0,-6.0,0.0,2.0
86902579,2017-08-06,54,2106464,0,33.0,-8.0,0.0,2.0
86902580,2017-08-05,54,2106464,0,32.0,-9.0,0.0,2.0
86902581,2017-08-04,54,2106464,0,31.0,-10.0,0.0,2.0
86902582,2017-08-02,54,2106464,0,29.0,-12.0,0.0,2.0
86902583,2017-08-01,54,2106464,0,28.0,-13.0,0.0,1.0
86902584,2017-07-31,54,2106464,0,27.0,-14.0,0.0,0.0
86902585,2017-07-30,54,2106464,0,26.0,-15.0,0.0,0.0


In [23]:
all_days.set_index('date',inplace=True)
bwd_days = all_days[['holiday']].sort_index().rolling(7,min_periods=1).sum().reset_index()
fwd_days = all_days[['holiday']].sort_index(ascending=False).rolling(7,min_periods=1).sum().reset_index()
all_days.reset_index(inplace=True)

In [24]:
print(bwd_days.head())
print(fwd_days.tail())

         date  holiday
0  2014-12-31      1.0
1  2015-01-01      2.0
2  2015-01-02      3.0
3  2015-01-03      3.0
4  2015-01-04      3.0
           date  holiday
970  2015-01-04      1.0
971  2015-01-03      0.0
972  2015-01-02      1.0
973  2015-01-01      2.0
974  2014-12-31      3.0


In [25]:
all_days = all_days.merge(bwd_days,on='date',how="left",suffixes=['','_bw'],copy=False)
del bwd_days
all_days = all_days.merge(fwd_days,on='date',how="left",suffixes=['','_fw'],copy=False)
del fwd_days
all_days

,date,holiday,Afterholiday,Beforeholiday,holiday_bw,holiday_fw
0,2017-08-31,0,7,-28.0,0.0,0.0
1,2017-08-30,0,6,-29.0,1.0,0.0
2,2017-08-29,0,5,-30.0,1.0,0.0
3,2017-08-28,0,4,-31.0,1.0,0.0
4,2017-08-27,0,3,-32.0,1.0,0.0
5,2017-08-26,0,2,-33.0,1.0,0.0
6,2017-08-25,0,1,-34.0,1.0,0.0
7,2017-08-24,1,0,0.0,1.0,1.0
8,2017-08-23,0,8,-1.0,0.0,1.0
9,2017-08-22,0,7,-2.0,0.0,1.0


In [26]:
print(df.head())
print(all_days.head())
gc.collect()

         date  store_nbr  item_nbr  onpromotion  Afteronpromotion  \
0  2017-08-10          1     96995            0               0.0   
1  2017-08-07          1     96995            0               0.0   
2  2017-08-06          1     96995            0               0.0   
3  2017-08-03          1     96995            0               0.0   
4  2017-08-02          1     96995            0               0.0   

   Beforeonpromotion  onpromotion_bw  onpromotion_fw  
0                0.0             0.0             0.0  
1                0.0             0.0             0.0  
2                0.0             0.0             0.0  
3                0.0             0.0             0.0  
4                0.0             0.0             0.0  
         date  holiday  Afterholiday  Beforeholiday  holiday_bw  holiday_fw
0  2017-08-31        0             7          -28.0         0.0         0.0
1  2017-08-30        0             6          -29.0         1.0         0.0
2  2017-08-29        0     

716

In [27]:
#df.to_csv('results/onpromotion_features.csv')
import pickle
pickle.dump(df, open('results/onpromotion_features.pickle', 'wb'))

In [28]:
#del all_days_cp
gc.collect()

0

In [29]:
def is_salary_day(date):
    return 1 if (date.day == 15 or calendar.monthrange(date.year,date.month)[1]==date.day) else 0
train['salary_day'] = train.date.apply(is_salary_day)

In [30]:
train.salary_day.unique()
gc.collect()
all_days = all_days[all_days.date.isin(train.date.unique())]

150

In [ ]:
joined = train.merge(all_days,on='date',how='left',copy=False)
joined = joined.merge(df,on=['store_nbr','item_nbr','date'],copy=False)
joined = joined.merge(stores,on='store_nbr',how="left",copy=False)
joined = joined.merge(items,on='item_nbr',how="left",copy=False)
joined = joined.merge(holidays_events,on='date',how="left",copy=False)
joined = joined.merge(oil,on="date",how="left",copy=False)
joined = joined.merge(transactions,on=['date','store_nbr'],how="left",copy=False)
print(joined.shape)
print(joined.head())
gc.collect()


In [ ]:
def add_datepart(df,column_name):
    df["Year"] = df[column_name].dt.year
    df["Month"] = df[column_name].dt.month
    df["Week"] = df[column_name].dt.week
    df["Day"] = df[column_name].dt.day
joined = add_datepart(joined)
#joined.to_csv('results/joined.csv')
pickle.dump(joined, open('results/joined.pickle', 'wb'))

In [ ]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler


In [ ]:

cat_var_dict = {'store_nbr': 10, 'item_nbr': 50, 'Year': 2, 'Month': 6,
'Day': 10, 'Week': 2,'holiday': 3, 'onpromotion': 3, 'salary_day':3,
'city': 6, 'state': 5, 'type_x': 3, 'cluster': 5, 
'family': 10, 'class': 20, 'perishable' :3,'type_y':3,
'onpromotion_bw': 1, 'onpromotion_fw': 1,
'holiday_bw': 1, 'holiday_fw': 1}

contin_vars =['Afterholiday','Beforeholiday','Afteronpromotion','Beforeonpromotion']

In [ ]:
cat_vars = [o[0] for o in 
            sorted(cat_var_dict.items(), key=operator.itemgetter(1), reverse=True)]

In [ ]:
for v in contin_vars: joined.ix[joined[v].isnull(), v] = 0
for v in cat_vars: joined.ix[joined[v].isnull(), v] = ""

In [ ]:
cat_maps = [(o, LabelEncoder()) for o in cat_vars]
contin_maps = [([o], StandardScaler()) for o in contin_vars]

In [ ]:
cat_mapper = DataFrameMapper(cat_maps)
cat_map_fit = cat_mapper.fit(joined)
cat_cols = len(cat_map_fit.features)
cat_cols

In [ ]:
contin_mapper = DataFrameMapper(contin_maps)
contin_map_fit = contin_mapper.fit(joined)
contin_cols = len(contin_map_fit.features)
contin_cols

In [ ]:
pickle.dump(contin_map_fit, open('results/contin_maps.pickle', 'wb'))
pickle.dump(cat_map_fit, open('results/cat_maps.pickle', 'wb'))